# GradientBoost 분류 기본코드

Gradient Boosting은 여러 개의 약한 예측 모형들을 결합하여 강력한 예측 모형을 만드는 앙상블 기법 중 하나.

기본 아이디어는 앞서 학습된 모델의 오류를 줄이는 방향으로 새로운 모델을 계속 추가해 나가는 것임

모델을 순차적으로 학습시키며, 각 모델은 이전 모델의 잔차를 줄이는 방향으로 학습한다.

이때, 각 단계에서 오류를 줄이기 위해 그래디언트 하강법을 사용하여 손실 함수(loss function)의 그래디언트(경사)를 계산하고, 이를 이용해 모델을 업데이트한다.

즉, 잔차를 이용하여 이전 모형의 약점을 보완하는 새로운 모형을 순차적으로 적합한 뒤 이들을 선형 결합하여 얻어진 모형을 생성한다.


| 특징 | 내용                                                                                                                          |
| ---- | ----------------------------------------------------------------------------------------------------------------------------- |
| 장점 | 1. 구현이 쉽다<br/>2. 정확도가 좋다.<br/>3.유연하다(의사결정나무 이외에 다른 알고리즘 적용 가능. 여러가지 손실함수 적용 가능) |
| 단점 | 1. 과적합 발생 가능성 큼<br/>2. 메모리 사용량이 큼<br/>3. 해석이 어려움                                                       |

GBM은 과적합에도 강한 뛰어난 예측 성능을 가진 알고리즘이지만 
드웨어 요구사항이 다른 알고리즘에 비해 높기 때문에 메모리 소비가 크고, 수행 시간이 오래 걸린다는 단점이 있어서 실제로 잘 사용되지는 않는다.

이 알고리즘이 처음 소개되고 이를 기반으로한 많은 알고리즘이 나왔으며 최근 가장 각광 받고 있는 ML 패키지는 `XGBoost`와 `LightGBM`, `CatBoost`이다.

## #01. 준비작업

### [1] 패키지 가져오기

In [ ]:
# 연결된 모듈이 업데이트 되면 즉시 자동 로드함
%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.plot import *
from hossam.analysis import *
from hossam.classification import *

# GradientBoost
from sklearn.ensemble import GradientBoostingClassifier

## #02. 데이터 가져오기

In [ ]:
origin = my_read_excel(
    "https://data.hossam.kr/mldata/pima_indians_diabetes.xlsx",
    categories=["Outcome"],
    info=False,
)

### [3] 전처리

1. `0` 값을 포함해서는 안되는 변수에 대해 평균으로 대체
2. 훈련/검증 데이터 분할 및 데이터 표준화

In [ ]:
# 정제 대상 필드명
zero_features = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# 0값을 결측치로 대체후 평균으로 다시 치환
df = origin.copy()
df[zero_features] = df[zero_features].replace(0, np.nan)
df2 = my_replace_missing_value(df)

# 훈련/검증 데이터 분할 및 데이터 표준화
x_train, x_test, y_train, y_test = my_train_test_split(df2, "Outcome", scalling=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

## #02. GradientBoost

### [1] 기본 모형 만들기

| 하이퍼 파라미터       | 기본값       | 설명                                                  | 추천사항                                   |
|-------------------|-----------|-----------------------------------------------------|-----------------------------------------|
| **`loss`**            | `log_loss`| 손실 함수. `log_loss`는 로지스틱 회귀와 유사.                | `exponential`는 AdaBoost와 유사  |
| **`learning_rate`**   | `0.1`     | 학습률. 각 트리의 기여도 조절.                                  | 낮은 값 + 높은 `n_estimators` 조합 권장.        |
| **`n_estimators`**    | `100`     | 부스팅 단계 수. 트리의 개수.                                 | 너무 높으면 과적합. 적절한 값 탐색 필요.            |
| **`subsample`**       | `1.0`     | 훈련에 사용하는 샘플 비율(`0~1 사이`). 스태깅 사용 시.                       | `1.0` 미만 값은 분산 감소 효과 있으나, 과적합 위험 감소. |
| **`criterion`**       | `friedman_mse` | 트리 품질 측정 기준. `friedman_mse`는 프리드먼의 평균 제곱 오차. | 기본값 권장.                             |
| `min_samples_split` | `2`       | 노드 분할 위한 최소 샘플 수.                                 | 과적합 방지를 위해 조정 가능.                      |
| `min_samples_leaf` | `1`       | 리프 노드가 되기 위한 최소 샘플 수.                            | 과적합 제어에 유용.                               |
| `max_depth`       | `3`       | 개별 회귀 추정기의 최대 깊이.                               | 깊이 크면 과적합 위험. 적절한 값 설정 중요.             |
| `max_features`    | `None`    | `‘sqrt’`, `‘log2’` 최적 분할을 위해 고려할 최대 특성 수.                         | 특성 수의 제곱근 사용 권장. 과적합 방지.               |
| `random_state`    | `None`    | 난수 시드.                                            | 결과 재현을 위해 사용.                          |


In [ ]:
gb = GradientBoostingClassifier(
        loss='log_loss',
        learning_rate=0.1,
        n_estimators=100,
        subsample=1,
        random_state=get_random_state())

gb.fit(X=x_train, y=y_train)

my_classification_result(estimator=gb, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)

my_classification_report(estimator=gb, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)